In [ ]:
from everywhereml.data import Dataset
from everywhereml.data.collect import SerialCollector
from everywhereml.preprocessing import Pipeline, MinMaxScaler, Window, SpectralFeatures
from everywhereml.sklearn.ensemble import RandomForestClassifier
from pprint import pprint
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Add the port your Arduino is connected to

Collect Training Data

In [ ]:
# port = '/dev/cu.usbmodem141401'
port = 'COM7'

try:
    imu_dataset = Dataset.from_csv(
        'imu.csv', 
        name='ContinuousMotion', 
        target_name_column='target_name'
    )
    
except FileNotFoundError:
    imu_collector = SerialCollector(
        # port='/dev/cu.usbmodem141401', 
        port=port, 
        baud=115200, 
        start_of_frame='IMU:', 
        feature_names=['ax', 'ay', 'az', 'gx', 'gy', 'gz']
    )
    imu_dataset = imu_collector.collect_many_classes(
        dataset_name='ContinuousMotion', 
        duration=30
    )
    
    # save dataset to file for later use
    imu_dataset.df.to_csv('imu.csv', index=False)

print(imu_dataset.df.columns)
print(imu_dataset.df['target_name'].value_counts())

## Classes to Use

In [ ]:
# This dictionary is used to enable/disable certain gestures
# comment out unwanted gestures
classes = [
    'clockwise_wrist_turn',
    'counterclockwise_wrist_turn',
    'palm_down_neutral',
    'palm_down_left_swipe',
    'palm_down_up',
    'palm_down_down',
    'palm_left_neutral',
    'palm_left_classic_slap',
    'palm_left_backhand_slap',
    'palm_left_up_chop',
    'palm_left_down_chop',
    'palm_down_right_swipe'
]

# Filter out gestures that are not enabled
imu_dataset.df = imu_dataset.df[imu_dataset.df['target_name'].isin(classes)]

imu_dataset.df['target_name'].value_counts()

In [ ]:
# this is the frequency of your sensor
# change according to your hardware
sampling_frequency = 104
mean_gesture_duration_in_millis = 1000
window_length = sampling_frequency * mean_gesture_duration_in_millis // 1000

imu_pipeline = Pipeline(name='ContinousMotionPipeline', steps=[
    MinMaxScaler(),
    # shift can be an integer (number of samples) or a float (percent)
    Window(length=window_length, shift=0.3),
    # order can either be 1 (first-order features) or 2 (add second-order features)
    SpectralFeatures(order=2)
])

pprint(imu_pipeline['SpectralFeatures'][0].feature_names)

"""
Apply feature pre-processing
"""
imu_dataset.apply(imu_pipeline)
imu_dataset.describe()

In [ ]:
"""
Plot features pairplot after feature extraction
Now it will start to make sense
Since SpectralFeatures generates 8 or 20 features (depending on the order)
for each axis, we limit the visualization to a more reasonable number
"""
imu_dataset.plot.features_pairplot(n=300, k=6)

In [ ]:
"""
Perform classification with a RandomForest
"""
imu_classifier = RandomForestClassifier(n_estimators=20, max_depth=20)
imu_train, imu_test = imu_dataset.split(test_size=0.3)
imu_classifier.fit(imu_train)

print('Score on test set: %.2f' % imu_classifier.score(imu_test))


# Plot confusion matrix
# If any off-diagonal values are dark, it means the gestures are too similar
# and the classifier is confusing them
df = imu_test.df
y_true = imu_test.df['target'].astype(int)
y_pred = imu_classifier.predict(imu_test.df.drop(columns=['target_name', 'target']))

mat = confusion_matrix(y_true, y_pred)
target_map = {i: name for i, name in enumerate(imu_test.target_names)}
plt.xticks(list(target_map.keys()), list(target_map.values()), rotation=90)
plt.yticks(list(target_map.keys()), list(target_map.values()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.imshow(mat, cmap='Blues', interpolation='nearest')
plt.show()
# round printed values to 2 decimal places
np.set_printoptions(precision=2)
print(mat / mat.max())

In [ ]:
"""
Port pipeline to C++
"""
print(imu_pipeline.to_arduino_file(
    '../Arduino/IMUClassify/Pipeline.h', 
    instance_name='pipeline'
))

# Pipeline.h file has a typo. Correct this
replacement_lines = [
    "\t\t\t\t\tstep0.transform(X)\n",
    "\t\t\t\t\t\n",
    "\t\t\t\t\t&& step1.transform(X)\n",
    "\t\t\t\t\t\n",
    "\t\t\t\t\t&& step2.transform(X)\n",
]

pipeline_file = '../Arduino/IMUClassify/Pipeline.h'

with open(pipeline_file, 'r') as f:
    file_data = f.readlines()

    with open(pipeline_file, 'w') as f:
        for i, line in enumerate(file_data):
            if i >= 262 and i < 267:
                line = replacement_lines[i-262]
            f.write(line)    


"""
Port classifier to C++
"""
print(imu_classifier.to_arduino_file(
    '../Arduino/IMUClassify/Classifier.h', 
    instance_name='forest', 
    class_map=imu_dataset.class_map
))

In [ ]:
df = pd.read_csv('imu_old.csv')
df['target'].unique()